In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from utils.physics.sound_model import HomogeneousSoundModel, MonthlyGridSoundModel, MONTHS_NAMES

In [ ]:
velocities_file = "../../../data/geo/velocities_grid.pkl"
compute_new = True

if compute_new:
    temp = [f"/media/plerolland/T7_PYR/WOA18/woa18_decav_t{i:02d}_04.nc" for i in range(1,13)]
    sal = [f"/media/plerolland/T7_PYR/WOA18/woa18_decav_s{i:02d}_04.nc" for i in range(1,13)]
    wanted_depths = [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0, 275.0, 300.0, 325.0, 350.0, 375.0, 400.0, 425.0, 450.0, 475.0, 500.0, 550.0, 600.0, 650.0, 700.0, 750.0, 800.0, 850.0, 900.0, 950.0, 1000.0, 1050.0, 1100.0, 1150.0, 1200.0, 1250.0, 1300.0, 1350.0, 1400.0, 1450.0, 1500.0, 2000.0, 2500.0, 3000.0, 3500.0, 4000.0, 4500.0, 5000.0]
    model_grid = MonthlyGridSoundModel(temperatures_files=temp, salinities_files=sal,  
                                       lat_bounds=[-72, 25], lon_bounds=[0, 180], depths=wanted_depths)
    model_grid.save_profiles(velocities_file)
else:
    model_grid = MonthlyGridSoundModel(profiles_checkpoint=velocities_file,
                                       lat_bounds=[-72, 25], lon_bounds=[0, 180])
model_homogeneous = HomogeneousSoundModel()

In [ ]:
m = 6

v = model_grid.velocities[m]
points = []
velocities = []
for i, lat in enumerate(model_grid.lat):
    for j, lon in enumerate(model_grid.lon):
        points.append([lat, lon])
        velocities.append(v[0, i, j])
        #velocities.append(model_grid.depths[np.argmin(v[:, i, j])])

points = np.array(points)

plt.scatter(points[:, 1], points[:, 0], s=1, marker='x', cmap='viridis', c=velocities)
plt.colorbar()

In [ ]:
lat, lon = -37, 78
model_grid.save_profile(lat, lon, f"monthly_vprofiles_{lat}_{lon}_woa18.txt", linterpolated=False)

In [ ]:
data, _, _ = model_grid.get_linterpolated_profile(lat, lon)
for i in range(12):
    plt.plot(data[i,:], -model_grid.depths, label=MONTHS_NAMES[i])
plt.legend()
plt.savefig("velocities.png")

In [ ]:
p1, p2 = (-12, 45), (-13, 45)

time_s_h = model_homogeneous.get_sound_travel_time(p1, p2)
time_s_g = model_grid.get_sound_travel_time(p1, p2)
print(f"Sound travel time estimation : {time_s_h} (homogeneous) and {time_s_g} (grid)")

env = model_grid.get_env(p1, p2, 0)

In [ ]:
import arlpy

arlpy.uwapm.plot_env(env)